In [1]:
import platform 
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("MyApp")
        .config("spark.sql.warehouse.dir","hdfs://localhost:9000/users/hive/warehouse")
        .config("spark.driver.memory","2g")
        .config("hive.metastore.uris", "thrift://localhost:9083")
        .enableHiveSupport()
        .getOrCreate())
    
    # spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark

JAVA_HOME is not set


RuntimeError: Java gateway process exited before sending its port number

In [8]:
spark.stop()

In [2]:
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [1]:
!hdfs dfs -put /usr/notebooks/Farmacias.csv /user/Daniel

In [3]:
!hdfs dfs -ls /user/Daniel/

Found 1 items
-rw-r--r--   1 Daniel root        758 2024-06-26 02:05 /user/Daniel/Farmacias.csv


In [2]:
spark.sql("create database silver").show()

++
||
++
++



In [4]:
df = spark.read.csv("/user/Daniel/Farmacias.csv",sep=";",header=True)
df.show()

+--------------------+-------------+-----------------------+---------------+
|              CIDADE|FLAG_SHOPPING|SEGUNDO_PRINCIPAL_SETOR|FATURAMENTO_TRI|
+--------------------+-------------+-----------------------+---------------+
|           São Paulo|          Sim|             Cosméticos|           34,8|
|            Campinas|          Sim|             Cosméticos|           28,6|
|           Guarulhos|          Sim|        Higiene pessoal|           27,3|
|              Osasco|          Sim|             Perfumaria|           24,5|
|São Bernardo do C...|          Sim|             Cosméticos|           24,1|
|      Ribeirão Preto|          Sim|             Cosméticos|           23,6|
|             Diadema|          Não|             Perfumaria|           21,8|
|            Sorocaba|          Sim|             Cosméticos|           19,8|
|        Praia Grande|          Não|        Higiene pessoal|           19,4|
|             Jundiaí|          Não|             Perfumaria|           18,5|

In [3]:
spark.sql('''

select 

--substring(UF_zi,1,2) as UF,
--count (distinct ano_cmpt),
--count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from bronze.rd_sih

--group by 1
--order by 1
''').show(30)

+--------------------------------------+--------+
|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+--------------------------------------+--------+
|                                299648|  299648|
+--------------------------------------+--------+



In [4]:
df = spark.read.csv('/DataSus/rd/csv',sep=";",header=True)
df.createOrReplaceTempView("test")

spark.sql('''

select 

--substring(UF_zi,1,2) as UF,
--count (distinct ano_cmpt),
--count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from test

--group by 1
--order by 1
''').show(30)

+--------------------------------------+--------+
|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+--------------------------------------+--------+
|                                299648|  345124|
+--------------------------------------+--------+



In [20]:
df = pd.DataFrame({'name': ['Raphael', 'Donatello'],
                   'mask': ['red', 'purple'],
                   'weapon': ['sai', 'bo staff']})


In [25]:

df.to_csv('df.csv')